**Tanish Chhabra<br>
101916038<br>
CS10**

In [1]:
import cupy as cp
import cudf
import cuml

In [2]:
train = cudf.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = cudf.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
Y = train['SalePrice']
train.drop('SalePrice',axis=1,inplace=True)

In [4]:
n_train,n_test = train.shape[0],test.shape[0]
train.shape,test.shape

In [5]:
df = cudf.concat([train,test],axis=0)
df.shape

In [6]:
# Drop columns having more than 25% missing values
to_drop = []
categorical_features = []
for col in df.keys():
    if df[col].dtype == 'object':
        categorical_features.append(col)
        
    if df[col].isnull().sum()>=len(train)//4:
        to_drop.append(col)

df = df.drop(to_drop,axis=1)

In [7]:
for col in df.keys():
    if df[col].isnull().sum()!=0:
        if col in categorical_features:
            df[col].fillna(df[col].mode()[0],inplace=True)
        else:
            df[col].fillna(df[col].mean(),inplace=True)

In [8]:
df.isnull().any().sum()

In [9]:
df = cudf.get_dummies(df)

In [10]:
train = df.iloc[:n_train,:]
test = df.iloc[n_test+1:,:]

In [11]:
train.shape,test.shape

In [12]:
X = train

In [13]:
X_train,X_validate,Y_train,Y_validate = cuml.train_test_split(X,Y,test_size=0.3,random_state=42)

In [14]:
algorithms = ['svd', 'eig', 'svd-qr', 'svd-jacobi']
for algo in algorithms:
    model = cuml.LinearRegression(algorithm=algo)
    trained = model.fit(X_train,Y_train)
    Y_predict = trained.predict(X_validate)
    
    mse = cuml.metrics.regression.mean_squared_error(Y_validate.astype('int64'),Y_predict.astype('int64'))
    mae = cuml.metrics.regression.mean_absolute_error(Y_validate.astype('int64'),Y_predict.astype('int64'))
    r2 = cuml.metrics.regression.r2_score(Y_validate.astype('float64'),Y_predict.astype('float64'))
    
    print("Algorithm: "+algo)
    print("R2 score:",str(r2),"      MSE:",str(mse),"       MAE:",str(mae))
    print()

In [15]:
model = cuml.LinearRegression(algorithm='svd-jacobi')
trained = model.fit(X,Y)
Y_predict = trained.predict(test)

In [16]:
Y_predict

In [17]:
output = cudf.DataFrame({'Id': test.Id, 'SalePrice': Y_predict})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

In [18]:
output